# Understanding Kiva

Contents

 - [ Relative gaps between funded and loaned amounts](#id-relative)
 - [Time to funding](#id-time)
 - [Time trends](#id-timetrends)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
loansdf = pd.read_csv('../input/kiva_loans.csv')
df2 = pd.read_csv('../input/loan_themes_by_region.csv')
df3 = pd.read_csv('../input/kiva_mpi_region_locations.csv')
df4 = pd.read_csv('../input/loan_theme_ids.csv')
loansdf.describe(include = 'all').T

<div id='id-relative'/>
## Relative gaps between funded and loaned amounts
The funded_amount from Kiva is 
> the amount of the loan which has been purchased by Kiva lenders. When this amount becomes equal to the loan_amount the loan moves to a funded state (soon followed by in_repayment). 

Therefore, Kiva - and its funders - might be concerned about loans that lenders are reluctant to purchase. I rank these loans by the percentage gap between the funded and loaned amounts, i.e. $\frac{loan\_amount - funded\_amount}{loan\_amount}$ 

Now this is very exploratory analysis, and it might just mean these same loans take a longer time to get a willing buyer. 

In [ ]:
loansdf['delta_amount'] = (loansdf.loan_amount - loansdf.funded_amount)/loansdf.loan_amount 
loansdf.loc[(loansdf.delta_amount > 0),'delta_amount'].hist().set_title('Distribution of relative gaps between funding and loaning')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('dark')
fg2, ax2 = plt.subplots(nrows = 1, ncols = 2, figsize = (20, 5))
countrydeltas = loansdf.groupby('country').mean().sort_values('delta_amount')
sns.barplot(countrydeltas.iloc[-10:,:].delta_amount, countrydeltas.iloc[-10:,:].index, ax = ax2[0]).set_title('10 countries with the largest relative gaps between funded and loaned amounts')
regiondeltas = loansdf.groupby('region').mean().sort_values('delta_amount')
sns.barplot(regiondeltas.iloc[-20:,:].delta_amount, regiondeltas.iloc[-20:,:].index, ax = ax2[1]).set_title('20 regions with the largest relative gaps between funded and loaned amounts')
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
fg1, ax1 = plt.subplots(nrows = 1, ncols = 2, figsize = (20,5))
sectordeltas = loansdf.groupby('sector').mean().sort_values('delta_amount')
sns.barplot(sectordeltas.delta_amount, sectordeltas.index, ax = ax1[0]).set_title('Relative gaps between funded and loaned amounts by sector')
activitydelta = loansdf.groupby('activity').mean().sort_values('delta_amount')
sns.barplot(activitydelta.iloc[-10:,:].delta_amount, activitydelta.iloc[-10:,:].index, ax = ax1[1]).set_title('10 activities with the largest relative gaps between funded and loaned amounts')
plt.tight_layout()

In [ ]:
repaymentdelta = loansdf.groupby('repayment_interval').mean().sort_values('delta_amount')
sns.barplot(repaymentdelta.delta_amount, repaymentdelta.index).set_title('Relative gaps between funded and loaned amounts by repayment interval')

As might be expected, Kiva lenders have no problems buying loans that are payable on a weekly basis, but for longer repayment intervals - the extreme being the 'bullet' case - Kiva lenders become less willing to bite the bullet.  

 <div id='id-time'/> 
 ## Time to funding
  - The vast majority of Kiva loans are funded within 100 days of posting. 
  - A lot of the disbursion happens **before** the funding of the loans. 
 

In [ ]:
loansdf['dtposted_time'] = pd.to_datetime(loansdf.posted_time)
loansdf['dtdisbursed_time'] = pd.to_datetime(loansdf.disbursed_time)
loansdf['dtfunded_time'] = pd.to_datetime(loansdf.funded_time)

In [ ]:
loansdf['fundeddelta'] = loansdf['dtfunded_time'] - loansdf['dtposted_time']
loansdf['disburseddelta'] = loansdf['dtdisbursed_time'] - loansdf['dtposted_time']

In [ ]:
fg3, ax3 = plt.subplots()
sns.distplot(loansdf.loc[:, 'fundeddelta'].dropna().astype('timedelta64[D]'), ax = ax3, label = 'Funding delta')
sns.distplot(loansdf.loc[:, 'disburseddelta'].dropna().astype('timedelta64[D]'), ax = ax3, label = 'Disbursion delta')
plt.legend()
plt.title('Time difference between funding/disbursion of loans and posting of loans')
plt.xlabel('Days')

<div id="id-timetrends"/>
## Time trends 
It might be interesting to look at time trends in Kiva's borrowing behaviour. There may be strong trends and seasonality in borrowing and posting patterns. 

NOTE: The date variable used here is 'Date'. 

The following analysis is done based on code in https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/

In [ ]:
loansdf['Date'] = pd.to_datetime(loansdf['date'], format = '%Y-%m-%d')
loansdf2 = loansdf.set_index('Date')
daily = loansdf2.loc[loansdf2.index.year < 2017,:].resample('M').mean()
fg, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,5))
ax[0].plot(daily.index, daily.funded_amount)
ax[0].set_title('Daily average funded amount')
monthly = loansdf2.resample('M').mean()
ax[1].plot(monthly.index, monthly.funded_amount)
ax[1].set_title('Monthly average funded amount')

From the monthly averaged amounts it looks like 2017 values are incomplete - August 2017 average seems a lot lower! I eliminate all values from 2017 onwards:

In [ ]:
from random import randrange
from matplotlib import pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(daily.funded_amount, model='additive')
result.plot()
plt.show()

There's trend and seasonality in the average loan disbursion. However, there isn't a clear time trend in the **total** loan disbursion over time (not shown). 

There is a peak in average loan disbursions at the start of each year. To make it clearer, I plot quarterly average loan disbursions: 

In [ ]:
quarterly = loansdf2.resample('Q').mean()
plt.plot(quarterly[quarterly.index.year < 2017].funded_amount)
plt.title('Quarterly average disbursions')

I suppose the strong seasonality may come from Kiva funders and borrowers alike wanting to fulfil their New Year resolutions! To support this hypothesis, I might do a count of loans: 

In [ ]:
dailycount = loansdf2.resample('M').count()
fg, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,5))
ax[0].plot(dailycount.index, dailycount.funded_amount)
ax[0].set_title('Daily count of loans')
monthlycount = loansdf2.resample('M').count()
ax[1].plot(monthlycount.index, monthlycount.funded_amount)
ax[1].set_title('Monthly count of loans')

So it doesn't look like people are borrowing more frequently at the start of the year - if anything there are troughs in the counts of loans at the start of the year. Perhaps, it is more that those who seek loans at the start of the year are more ambitious, and therefore borrow more on average. As shown below, there's seasonality in the median loans too! 

In [ ]:
dailymead = loansdf2.resample('M').median()
fg, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,5))
ax[0].plot(dailymead.index, dailymead.funded_amount)
ax[0].set_title('Daily median loans')
monthlymead = loansdf2.resample('M').median()
ax[1].plot(monthlymead.index, monthlymead.funded_amount)
ax[1].set_title('Monthly median loans')